# shp 파일 인코딩 문제 해결 및 json 으로 변환
- geoPandas 패키지 설치
- 데이터 추가 전처리(중복 문제 해결, cx, cy 항목 정제, column name 수정, 상태 항목 제거)
- 최종본 json으로 출력

In [1]:
import geopandas as gpd
import pandas as pd

In [6]:
df = gpd.read_file('./20220416.shp', encoding='euc-kr')
df[:3]

,번호,식당명,유형,주메뉴,위치,전화번호,홈페이지,지정기관,cx,cy,경도,geometry
0,133,서리원,한식,한정식,"서울시 마포구 공덕동 463 현대하이엘 지1층 123,124,125호",02)6353-1447,-,마포구위생과지정,0,0,정좌표,POINT (126.95437 37.54980)
1,81,태능식당,한식,돼지갈비,서울시 마포구 굴레방로 19-1,365-1595,http://place.map.daum.net/8554216,모범음식점,0,0,정좌표,POINT (126.95551 37.55587)
2,72,군자네,한식,고등어김치찜,"서울시 마포구 대흥로 53, 1층",706-5454,http://place.map.daum.net/15963187,모범음식점,0,0,정좌표,POINT (126.93953 37.54554)


In [7]:
geo = df['geometry']
geo[:10]

0    POINT (126.95437 37.54980)
1    POINT (126.95551 37.55587)
2    POINT (126.93953 37.54554)
3    POINT (126.93953 37.54554)
4    POINT (126.94038 37.54586)
5    POINT (126.94867 37.53998)
6    POINT (126.94876 37.53979)
7    POINT (126.94778 37.53984)
8    POINT (126.94823 37.54008)
9    POINT (126.94853 37.54002)
Name: geometry, dtype: geometry

In [11]:
df['cx'] = df['geometry'].x
df['cy'] = df['geometry'].y

In [14]:
df[:3]

,번호,식당명,유형,주메뉴,위치,전화번호,홈페이지,지정기관,cx,cy,경도,geometry
0,133,서리원,한식,한정식,"서울시 마포구 공덕동 463 현대하이엘 지1층 123,124,125호",02)6353-1447,-,마포구위생과지정,126.954374,37.549803,정좌표,POINT (126.95437 37.54980)
1,81,태능식당,한식,돼지갈비,서울시 마포구 굴레방로 19-1,365-1595,http://place.map.daum.net/8554216,모범음식점,126.955512,37.555869,정좌표,POINT (126.95551 37.55587)
2,72,군자네,한식,고등어김치찜,"서울시 마포구 대흥로 53, 1층",706-5454,http://place.map.daum.net/15963187,모범음식점,126.939532,37.545542,정좌표,POINT (126.93953 37.54554)


In [16]:
# 중복항목 제거
# ex: df2 = df.apply(lambda x: x.astype(str).str.lower()).drop_duplicates(subset=['Courses', 'Fee'], keep='first')
total = df.drop_duplicates(subset=['식당명'], keep='first')
total.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 239 entries, 0 to 366
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   번호        239 non-null    object  
 1   식당명       239 non-null    object  
 2   유형        239 non-null    object  
 3   주메뉴       238 non-null    object  
 4   위치        239 non-null    object  
 5   전화번호      237 non-null    object  
 6   홈페이지      239 non-null    object  
 7   지정기관      239 non-null    object  
 8   cx        239 non-null    float64 
 9   cy        239 non-null    float64 
 10  경도        239 non-null    object  
 11  geometry  239 non-null    geometry
dtypes: float64(2), geometry(1), object(9)
memory usage: 24.3+ KB


In [19]:
# geopandas df to pandas df
pdf = pd.DataFrame(total)

In [21]:
# geometry column 없에기

pdf.drop('geometry', inplace=True, axis=1)

In [24]:
pdf[:2]

,번호,식당명,유형,주메뉴,위치,전화번호,홈페이지,지정기관,cx,cy,경도
0,133,서리원,한식,한정식,"서울시 마포구 공덕동 463 현대하이엘 지1층 123,124,125호",02)6353-1447,-,마포구위생과지정,126.954374,37.549803,정좌표
1,81,태능식당,한식,돼지갈비,서울시 마포구 굴레방로 19-1,365-1595,http://place.map.daum.net/8554216,모범음식점,126.955512,37.555869,정좌표


In [25]:
# column names replacement
pdf.columns = ['id', 'name', 'type', 'main_menu', 'address', 'number', 'website', 'organization', 'cx', 'cy', 'coord_state']

In [26]:
pdf

,id,name,type,main_menu,address,number,website,organization,cx,cy,coord_state
0,133,서리원,한식,한정식,"서울시 마포구 공덕동 463 현대하이엘 지1층 123,124,125호",02)6353-1447,-,마포구위생과지정,126.954374,37.549803,정좌표
1,81,태능식당,한식,돼지갈비,서울시 마포구 굴레방로 19-1,365-1595,http://place.map.daum.net/8554216,모범음식점,126.955512,37.555869,정좌표
2,72,군자네,한식,고등어김치찜,"서울시 마포구 대흥로 53, 1층",706-5454,http://place.map.daum.net/15963187,모범음식점,126.939532,37.545542,정좌표
4,155,개성손만두,한식,만두전골,서울시 마포구 대흥로 64,02-701-2500,-,마포구위생과지정,126.940378,37.545857,정좌표
5,158,동해세수대야 동태탕 마포역점,한식,동태탕,서울시 마포구 도화2안길 2-4,02-707-1011,-,마포구위생과지정,126.948669,37.539983,정좌표
...,...,...,...,...,...,...,...,...,...,...,...
354,97,거부,한식,소등심,서울시 마포구 토정로 37길 46,717-9387,http://place.map.daum.net/12085164,모범음식점,126.947372,37.542558,정좌표
358,101,청춘구락부,한식,곱창,서울시 마포구 토정로308,702-1399,http://place.map.daum.net/21034604,모범음식점,126.942905,37.540088,정좌표
360,104,7%칠백식당,한식,삼겹살,서울시 마포구 토정로32길 4,02-718-0705,http://place.map.daum.net/27316286,모범음식점,126.941284,37.541070,정좌표
364,63,신원복집식당,일식,복,서울시 마포구 홍익로 29,333-1977,http://place.map.daum.net/16819283,모범음식점,126.921787,37.554637,정좌표


In [27]:
# 내보내기

pdf.to_json('./mapojoy_data.json', orient='records', force_ascii=False)